# Solving Linear Equations



## Motivation

Linear algebra is a fundamental tool in scientific computing. Linear systems of equations is one of the most common problems in scientific computing. We will use this module to introduce the basic concept, algorithms and tools for solving linear equations in Python.

## Basic Concepts

If you are given a set of linear equations, you can write it in a matrix form as

$$
A x = b
$$

where $A$ is a matrix, $x$ is a vector, and $b$ is a vector. $A$ is also called
the coefficient matrix, and $b$ is called the right-hand side vector. In linear
algebra, $A$ and $b$ are often given, and we need to find $x$. 

If the number of equations is equal to the number of unknowns, i.e., the size of
$A$ is $n\times n$, then $A$ is called a square matrix. If $A$ is a square
matrix and the determinant of $A$ is not zero (or say, the rows or columns are linearly
independent), then $A$ is called a nonsingular matrix, and the solution $x$ is
unique.



## Matrix Formulation

Let's start with a simple example:


$$
\begin{bmatrix}
2 & 1 & 4 \\
1 & 2 & 3 \\
3 & 1 & 2
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}
$$


In Python, the first thing to know is to use `numpy` to work with matrices and solve the equations.


In [1]:
import numpy as np


A = np.array([[2, 1, 4], [1, 2, 3], [3, 1, 2]])
b = np.array([1, 2, 3])


Some notes:

- It is very common to do `import numpy as np` at the beginning of a Python script or a Jupyter notebook.
- If you are familiar with MATLAB, you may recognize that `np.array([])` is the same as `[ ]` in MATLAB. It can be used to create an array (1D) or an array of arrays (2D).

### Solution by Inversion

The solution of the linear equations $A x = b$ is, mathematically, $x = A^{-1} b$. It can be implemented in Python as follows:

In [2]:
Ainv = np.linalg.inv(A)

x = Ainv @ b

print(x)

[ 0.90909091  1.36363636 -0.54545455]


Additional notes are:

- The `np.linalg.inv` function is used to compute the inverse of a matrix.
- The `@` operator is used for **matrix multiplication** in Python.

A quick cheatsheet can be found [here](https://cheatsheets.quantecon.org/index.html) for MATLAB users to get started with Python.





### Non-Inversion Method


But it is never a good idea to use the inverse of a matrix to solve the linear equations, because it can be 
- numerically unstable
- computationally expensive

For interested readers, here's a good writeup on [why one should never invert a matrix for Ax=b](https://gregorygundersen.com/blog/2020/12/09/matrix-inversion/).

In Python, we can use the `np.linalg.solve` function to solve the linear equations.


In [3]:
x = np.linalg.solve(A, b)

print(x)

[ 0.90909091  1.36363636 -0.54545455]


`np.linalg.solve` uses the [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition) to solve the equations. It needs to be understood that the actual decomposition is performed by an underlying library, LAPACK, written in C/Fortran. This is a good example of how Python can be used as a "glue" language to combine different libraries and offer an high-level interface.


## Sparse Linear Equations

A set of equations is called sparse if the $A$ matrix has many zero elements. When using sparse matrix types to store $A$, the linear equations can be solved with better efficiency and less memory usage.

Sparse matrices are common in power system applications. As is typical, a substation represented as a bus is only connected to a few other buses. If we write out the nodal equation $I = Y V$, the [admittance matrix](https://en.wikipedia.org/wiki/Nodal_admittance_matrix) $Y$ is sparse.

Let's use the numbers from the previous example but create a sparse matrix with `scipy`.



In [9]:
A = np.array([[2, 1, 4], [1, 2, 3], [3, 1, 2]])
b = np.array([1, 2, 3])

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

A_sparse = csr_matrix(A)

x = spsolve(A_sparse, b)

print(x)

[ 0.90909091  1.36363636 -0.54545455]


A few notes are:
- The sparse matrix is created by `csr_matrix` from a dense matrix $A$. This is not the most efficient way if the sparse matrix can be created without creating a full matrix. Here we use the dense matrix to illustrate the solution.
- `scipy.sparse.linalg.spsolve` is used to solve the sparse linear equations. The solution is the same as the dense matrix.

If we inspect `A_sparse`, we can see that it is a sparse matrix.



In [10]:
print(A_sparse)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 9 stored elements and shape (3, 3)>
  Coords	Values
  (0, 0)	2
  (0, 1)	1
  (0, 2)	4
  (1, 0)	1
  (1, 1)	2
  (1, 2)	3
  (2, 0)	3
  (2, 1)	1
  (2, 2)	2


SciPy supports many types of sparse matrices, including `csr_matrix`, `csc_matrix`, `coo_matrix`, `dia_matrix`, `dok_matrix`, `lil_matrix`. A summary can be found [here](https://docs.scipy.org/doc/scipy/reference/sparse.html#sparse-matrix-classes).

The differences are in the underlying storage for representing the matrix on a computer. For example, `coo_matrix` stores the matrix in a coordinate format, which is efficient for creating a sparse matrix from a dense matrix. Similarly, `dok_matrix` stores the matrix in a dictionary, which is also efficient for increamentally building a matrix. These two formats are not suitable for numerical operations because of the difficulty in accessing the elements in any desired order. Specifically, for example, to access `(1, 1)` in a `coo_matrix`, we need to search through all the stored elements.





